In [25]:
%load_ext dotenv
%dotenv
import os
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


Loading the pdf content into Document objects langchain can handle

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../data/raw/50.pdf"
loader = PyPDFLoader(file_path,mode="single")
docs = loader.load()
print(docs[0])

/Users/admin/Desktop/RAG-tutor/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/admin/Desktop/RAG-tutor/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  F

page_content='
Data Mining: The Textbook
Charu C. Aggarwal
Data Mining
The Textbook
Charu C. Aggarwal
IBM T.J. Watson Research Center
Yorktown Heights
New York
USA
A solution manual for this book is available on Springer.com.
ISBN 978-3-319-14141-1 ISBN 978-3-319-14142-8 (eBook)
DOI 10.1007/978-3-319-14142-8
Library of Congress Control Number: 2015930833
Springer Cham Heidelberg New York Dordrecht London
c⃝ Springer International Publishing Switzerland 2015
This work is subject to copyright. All rights are reserved by the Publisher, whether the whole or part of
the material is concerned, speciﬁcally the rights of translation, reprinting, reuse of illustrations, recitation,
broadcasting, reproduction on microﬁlms or in any other physical way, and transmission or information
storage and retrieval, electronic adaptation, computer software, or by similar or dissimilar methodology
now known or hereafter developed.
The use of general descriptive names, registered names, trademarks, servic

Splitting text into smaller chunks

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
texts = text_splitter.split_documents(docs)
len(texts)


Loading an embedding model based on env setup

In [28]:
from src.ingest.embeddings import get_embeddings
embedding_mode = os.getenv("EMBEDDING_MODE","local")

embeddings = get_embeddings(embedding_mode,model_name="qwen3-embedding:0.6b")
print(embeddings)

RuntimeError: Failed to query Ollama models

We need a vectorstore to save the embeddings

In [17]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

The embedding model is used to generate an embedding for each document before they are saved to the vectorstore

In [18]:
document_ids = vector_store.add_documents(documents=texts)
    
print(document_ids[:3])

['e433f72d-cd1c-4cad-a277-15f93994ec9e', 'd37d4315-8f32-4faa-984f-f0f3d4003e35', '0cc0d4c6-4deb-45eb-8bd7-3d3361d15eca']


Similarity search creates an embedding of the question and then returns documents form the vectorstore with the most similar embeddings

In [ ]:
vector_store.save_local("../data/vector_db/simple_chunking")

new_vector_store = FAISS.load_local(
    "../data/vector_db/simple_chunking", embeddings, allow_dangerous_deserialization=True
)
res = vector_store.similarity_search("what chapters are there ih the book")
print(res)


[Document(id='0eaa8a8c-d189-45ed-90a9-1682b5165bd0', metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2026-02-18T10:10:38+00:00', 'source': '../data/raw/50.pdf', 'total_pages': 50}, page_content='of applications that arise in the context of those data types.\nSuggestions for the Instructor\nThe book was speciﬁcally written to enable the teaching of both the basic data mining and\nadvanced data mining courses from a single book. It can be used to oﬀer various types of\ndata mining courses with diﬀerent emphases. Speciﬁcally, the courses that could be oﬀered\nwith various chapters are as follows:\n•Basic data mining course and fundamentals: The basic data mining course\nshould focus on the fundamentals of data mining. Chapters 1, 2, 3, 4, 6, 8, and 10\ncan be covered. In fact, the material in these chapters is more than what is possible\nto teach in a single course. Therefore, instructors may need to select topics of their\ninterest from these chapter

4